<a href="https://colab.research.google.com/github/isbkch/algorithms/blob/master/Copy_of_Generating_AI_Photos_Of_You.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time

API_KEY = "51124f9d-c1db-4af1-b231-4b8871ede467"

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

IMAGES = [
    "https://scontent-lga3-2.xx.fbcdn.net/v/t39.30808-6/328469837_5898617053551086_2215835468682256760_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=09cbfe&_nc_ohc=SlZipNsVjhoAX-MIttP&_nc_ht=scontent-lga3-2.xx&oh=00_AfCqCN3gar75t0wzw0jmyQbpe8Trii75v8y5XxY2uOI2oQ&oe=641A1F35",
    "https://scontent-lga3-1.xx.fbcdn.net/v/t39.30808-6/322371548_1490776614764941_6022324307691385626_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=174925&_nc_ohc=CnfiKXpusRQAX_cEW1Z&_nc_ht=scontent-lga3-1.xx&oh=00_AfAuuh_Pfdfclau0KVjMGnTCCT8PBB2F6LCNfSXOBwsHmQ&oe=6419DEA1",
    "https://scontent-lga3-2.xx.fbcdn.net/v/t39.30808-6/313421341_105353449048885_4875932506936517365_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=174925&_nc_ohc=vp-vd6Az6dAAX-vZhv6&_nc_ht=scontent-lga3-2.xx&oh=00_AfASmdA3s9bcXU-kKkGZCpT_FamF8nu1tUSzzOzv17gKkA&oe=6419AF28"
]

In [ ]:
def create_model(title):
    url = "https://api.tryleap.ai/api/v1/images/models"

    payload = {
        "title": title,
        "subjectKeyword": "@me"
    }

    response = requests.post(url, json=payload, headers=HEADERS)

    model_id = json.loads(response.text)["id"]
    return model_id


def upload_image_samples(model_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/samples/url"

    payload = {"images": IMAGES}
    response = requests.post(url, json=payload, headers=HEADERS)


def queue_training_job(model_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/queue"
    response = requests.post(url, headers=HEADERS)
    data = json.loads(response.text)

    print(response.text)

    version_id = data["id"]
    status = data["status"]

    print(f"Version ID: {version_id}. Status: {status}")

    return version_id, status


def get_model_version(model_id, version_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/versions/{version_id}"
    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    version_id = data["id"]
    status = data["status"]

    print(f"Version ID: {version_id}. Status: {status}")

    return version_id, status


def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 50,
        "width": 512,
        "height": 512,
        "numberOfImages": 1,
        "seed": 4523184
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status


def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    image = None

    if len(data["images"]):
        image = data["images"][0]["uri"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, image




In [ ]:
# Let's create a custom model so we can fine tune it.
model_id = create_model("Sample")

# We now upload the images to fine tune this model.
upload_image_samples(model_id)

# Now it's time to fine tune the model. Notice how I'm continuously 
# getting the status of the training job and waiting until it's
# finished before moving on.
version_id, status = queue_training_job(model_id)
while status != "finished":
    time.sleep(10)
    version_id, status = get_model_version(model_id, version_id)



In [ ]:
# Now that we have a fine-tuned version of a model, we can
# generate images using it. Notice how I'm using '@me' to 
# indicate I want pictures similar to the ones we upload to 
# fine tune this model.
inference_id, status = generate_image(
    model_id, 
    prompt="A photo of @me with a tall black hat and sunglasses"
)
while status != "finished":
    time.sleep(10)
    inference_id, status, image = get_inference_job(model_id, inference_id)

print(image)